In [12]:
!apt update && apt install -y curl git
!curl -fsSL https://ollama.com/install.sh | sh
!ollama pull llama3.2

!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 8501/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries
OLLAMA_API_URL = "http://127.0.0.1:11434"
!nohup ollama serve > /dev/null 2>&1 &

!pip install streamlit PyPDF2 requests python-docx
!pip install pyngrok
from pyngrok import ngrok

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [13]:
# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "2tqZZ0zm7U5azTMJ6B7In96Eia0_7M44dRZx1KKTz8Dfd7R4Z"

# Set authentication token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok Auth Token Set!")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit App is running at: {public_url}")


✅ ngrok Auth Token Set!
🚀 Streamlit App is running at: https://48644e9d337d.ngrok-free.app


In [14]:
import requests

OLLAMA_API_URL = "http://127.0.0.1:11434/api/generate"

headers = {"Content-Type": "application/json"}
payload = {
    "model": "llama3.2",
    "prompt": "What is the capital of France?",
    "stream": False
}

response = requests.post(OLLAMA_API_URL, json=payload, headers=headers)

if response.status_code == 200:
    print("✅ Response from Ollama:", response.json()["response"])
else:
    print(f"❌ Error from Ollama: {response.status_code}, {response.text}")


❌ Error from Ollama: 404, {"error":"model 'llama3.2' not found"}


In [15]:
%%writefile app.py
import streamlit as st
import requests
import tempfile
import os
import tiktoken
from datetime import datetime
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
import random
import seaborn as sns

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

# Streamlit UI setup
st.set_page_config(page_title="Chat with Text File (Llama 3.2)", layout="wide")
st.title("📄 Chat with Your Text File (Llama 3.2)")
st.caption("Upload a text file and get automatic chat statistics!")

# Upload the file
uploaded_file = st.file_uploader("📂 Upload your text file", type=["txt"])

if uploaded_file:
    st.success(f"✅ Uploaded: {uploaded_file.name}")

    # Save the uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as temp_file:
        temp_file.write(uploaded_file.read())
        temp_txt_path = temp_file.name

    # Read the text file
    with open(temp_txt_path, "r", encoding="utf-8") as f:
        file_text = f.read()

    # Tokenize and display text information
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(file_text)
    st.write(f"Document length: {len(file_text)} characters.")
    st.write(f"Number of tokens: {len(tokens)}")

    # Remove the temporary file
    os.remove(temp_txt_path)

    # Display the text (first 1000 characters for preview)
    with st.expander("🔍 View Extracted Text"):
        st.text_area("Text Content", file_text[:1000] + "..." if len(file_text) > 1000 else file_text, height=200)

    def parse_chat(file_text):
      messages = []
      senders = set()  # To collect unique senders
      lines = file_text.split('\n')

      for line in lines:
        # Adjust regex to account for potential non-breaking spaces, emojis, and other special characters
        match = re.match(r"(\d{1,2}/\d{1,2}/\d{2,4}),\s*(\d{1,2}:\d{2}\s*(AM|PM))\s*-\s*(.*?):\s*(.*)", line)

        if match:
          date = match.group(1)
          time = match.group(2)
          sender = match.group(4)
          message = match.group(5)
          timestamp = f"{date} {time}"
          messages.append({"timestamp": timestamp, "sender": sender, "message": message})
          senders.add(sender)
        else:
          # Debugging: Display the line that is being skipped
          print(f"Skipping line: {line}")

      return messages, list(senders)


    # Get the parsed messages and participant names
    messages, senders = parse_chat(file_text)

    # Create a DataFrame from the messages
    df = pd.DataFrame(messages)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['day_of_week'] = df['timestamp'].dt.day_name()

    # Refined system prompt for Llama
    system_prompt = (
        "You are an AI assistant that answers questions based on the given document."
        "Give direct answers, provide numbers or percentages for statistics."
        "Here are some questions you need to answer based on the chat history:\n"
        "1. Give me the statistics data for the weekdays from the chat.\n"
        "2. Based on the timestamp, check for the immediate responses in the chat.\n"
        "3. Who uses more emojis in the chat?\n"
        "4. Analyze the entire chat history from the first recorded message to the last recorded message, without limiting the timeframe."
        "Identify the person who initiated the most conversations and provide a detailed breakdown, including the count of conversations started by each participant."
        "What is the average response time during conversations?\n"
        "5. Give me statistics of who focuses a lot on the chat?\n"
        "6. What is the overall count of the total number of messages sent by each participant?\n"
        "7. Top 3 most used words by each participant in the chat?\n"
        "8. How many times did each participant start the conversation?\n"
        "9. Who is likely to end the conversation first? Give me the count.\n"
        "10. Statistics of who used the emojis more in total?\n"
        "11. What is the total number of questions asked by each participant?\n"
        "Answer all the above questions based on the chat data provided. Each answer should be clear and concise."
        "You should also do sentiment analysis of the document uploaded, analyze emotions like 'sad','happy','anger','love','jealousy','care','disagreements','bored','funny','excitement' and other emotions."
    )

    # Max tokens for Llama 3.2 is 128,000 tokens
    max_tokens = 128000
    max_chars = max_tokens * 4

    # Split text into chunks of 5000 characters (or a reasonable limit)
    chunk_size = 5000
    chunks = [file_text[i:i + chunk_size] for i in range(0, len(file_text), chunk_size)]

    responses = []
    for chunk in chunks:
        ollama_payload = {
            "model": "llama3.2",
            "prompt": f"{system_prompt}\n\nDocument:\n{chunk}\n\nAI Answer:",
            "stream": False
        }

        response = requests.post(OLLAMA_URL, json=ollama_payload)

        if response.status_code == 200:
            result = response.json().get("response", "⚠️ No response from Analyzer.")
            responses.append(result)
        else:
            st.error("⚠️ Failed to get response from Analyzer. Is Ollama running?")
            break  # Stop further processing if there’s an error

    # Combine all responses
    final_response = "\n\n".join(responses) if responses else "⚠️ No response from Analyzer."

    # Replace placeholders with actual names dynamically
    for sender in senders:
        final_response = final_response.replace("participant", sender)

    # Display the answers
    st.subheader("📌 Automatic Chat Analysis Results:")
    st.write(final_response)



    #emoji usage count calculatiion
    emoji_usage = df['message'].apply(lambda x: sum([1 for c in x if c in "🙂😀😃😄😁😆😂🤣😊😎😍🥺🙄😛😜🤔😳👀🙄😑😐😬👍😌🙂"]))  # Adjust emoji detection logic
    emoji_count = emoji_usage.sum()

    #response time calculation
    df['response_time'] = df['timestamp'].diff().shift(-1)  # Calculate time between messages
    avg_response_time = df['response_time'].mean()

    #calculation of weekly activity activity chat
    def get_time_slot(hour):
      if 5 <= hour < 8:
        return "Early morn."
      elif 8 <= hour < 12:
        return "Morning"
      elif 12 <= hour < 17:
        return "Afternoon"
      elif 17 <= hour < 21:
        return "Evening"
      else:
        return "Night"

    df['hour']=df['timestamp'].dt.hour
    df['weekday']=df['timestamp'].dt.day_name().str[:3]
    df['time_slot']=df['hour'].apply(get_time_slot)

    activity_matrix = df.groupby(['time_slot', 'weekday']).size().unstack(fill_value=0)

    # Step 4: Order rows and columns
    slot_order = ["Early morn.", "Morning", "Afternoon", "Evening", "Late night"]
    day_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    activity_matrix = activity_matrix.reindex(index=slot_order, columns=day_order)




    # Dashboard Title
    st.markdown("## 📊 Chat Dashboard")

    # Row 1: Key Stats
    col1, col2, col3 = st.columns(3)
    with col1:
      st.metric("📈Total Messages", len(df))
    with col2:
      st.metric("😄Total Emojis", emoji_count)
    with col3:
      st.metric("⏳Avg. Response Time", str(avg_response_time).split('.')[0])

    # Row 2: Bar + Focus Pie
    col1, col2 = st.columns([2, 1])
    with col1:
      st.markdown("#### 📅 Messages by Day")
      day_stats = df['day_of_week'].value_counts()
      st.write(day_stats)
      fig, ax = plt.subplots(figsize=(3, 2))
      day_stats.plot(kind='bar', color='skyblue', ax=ax)
      ax.set_title('Messages Sent by Day of the Week', fontsize=10)
      ax.set_xlabel('Day of the Week',fontsize=8)
      ax.set_ylabel('Number of Messages',fontsize=8)
      ax.tick_params(axis='x',labelrotation=45,labelsize=7)
      ax.tick_params(axis='y',labelsize=7)
      plt.tight_layout(pad=0.5)
      st.pyplot(fig)

    with col2:
      st.markdown("#### 🎯 Focus Pie")
      focus_counts=df['sender'].value_counts()
      st.write(focus_counts)
      def small_autopct(pct):
        return f'{pct:.1f}%' if pct > 0 else ''
      fig, ax = plt.subplots(figsize=(1.8, 1.8))
      focus_counts.plot(kind='pie', ax=ax, autopct=small_autopct,colors=['#b38eda','#8ec5da'], textprops={'fontsize': 5})
      ax.set_ylabel('')
      ax.set_title('', fontsize=7)
      ax.set_position([0.1, 0.1, 0.8, 0.8])
      st.pyplot(fig)

    # Row 3: Sentiment Pie
    col1,col2=st.columns([1,2])
    with col1:
      st.markdown("#### 👻 Sentiment Breakdown")
      sentiments = ['happy', 'sad', 'angry', 'excited', 'bored','funny','love','care','disagreements']  # Placeholder, you can replace with actual sentiment analysis
      sentiment_counts = Counter([random.choice(sentiments) for _ in range(len(df))])
      st.write(sentiment_counts)

      fig, ax = plt.subplots(figsize=(2, 2))
      sentiment_counts=Counter([random.choice(sentiments) for _ in range(len(df))])
      sentiment_series=pd.Series(sentiment_counts)
      def small_autopct(pct):
        return f'{pct:.1f}%' if pct > 0 else ''
      sentiment_series.plot(kind='pie', ax=ax, autopct=small_autopct,colors=['#ff9999','#66b3ff','#99ff99','#ffcc99','#21aed8','#c292ef','#ef92c1','#b97e7e','#d1e377'],textprops={'fontsize': 5})
      ax.set_ylabel('')
      ax.set_title('sentiments', fontsize=7)
      ax.set_position([0.1, 0.1, 0.8, 0.8])
      st.pyplot(fig)

    with col2:
      st.markdown("#### ☀️🌔 weekly chat activity")
      fig,ax=plt.subplots(figsize=(7,3.5))
      sns.heatmap(activity_matrix, annot=True, fmt='.0f', cmap='YlOrBr', cbar=False, linewidths=0.5, linecolor='white', ax=ax)
      ax.set_xlabel("day of the week",fontsize=10)
      ax.set_ylabel("time of the day",fontsize=10)
      ax.set_title("chat data by time and day",fontsize=10,pad=10)

      ax.tick_params(axis='x',labelrotation=0,labelsize=8)
      ax.tick_params(axis='y',labelsize=8)
      st.pyplot(fig)




    # User's Question input
    question = st.text_input("💡 Ask a question about the document")

    if question:
        st.info("🤖 Sending question to Llama 3.2 :)...")

        # Refined system prompt to focus on direct answers only
        system_prompt_user = (
            "You are an AI assistant that answers questions based on the given document."
            "When asked about a specific message, return only the exact date and time when the message was sent."
            "If the message is not found or timestamped, respond with 'I don't know'."
            "Do not provide any additional context, reasoning, or elaboration."
            "Only return the direct date and time of the message. Do not include any other information."
            "Perform sentiment analysis on the document uploaded, analyze the emotions such as 'upset','anger','jealous','care','love','emotional','happiness','sad','funny', and others."
            "Provide accurate answers. Don't make things up."
            "If the question is not related to the document, politely respond that you are tuned only to answer questions that are related to the document."
        )

        # Max tokens for Llama 3.2 is 128,000 tokens
        max_tokens = 128000
        chunk_size = 5000  # Split text into chunks of 5000 characters (or a reasonable limit)
        chunks = [file_text[i:i + chunk_size] for i in range(0, len(file_text), chunk_size)]

        responses_user_question = []
        for chunk in chunks:
            ollama_payload_user_question = {
                "model": "llama3.2",
                "prompt": f"{system_prompt_user}\n\nDocument:\n{chunk}\n\nUser Question: {question}\n\nAI Answer:",
                "stream": False
            }

            response = requests.post(OLLAMA_URL, json=ollama_payload_user_question)

            if response.status_code == 200:
                result = response.json().get("response", "⚠️ No response from Analyzer.")
                responses_user_question.append(result)
            else:
                st.error("⚠️ Failed to get response from Analyzer. Is Ollama running?")
                break

        # Combine all responses for the user's question
        final_response_user_question = "\n\n".join(responses_user_question) if responses_user_question else "⚠️ No response from Analyzer"

        # Display the user's answer
        st.subheader("📌 User's Question Answer:")
        st.write(final_response_user_question)


Overwriting app.py


In [16]:
!pip install tiktoken
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.187.222:8501

  Stopping...
